In [199]:
import pandas as pd
import numpy as np

from CAPM.regression import rolling_regression
from regression import RollingRegressor

In [200]:
thirty_year_bond=pd.read_feather(r'C:\Users\m1309\Downloads\bond_data\bond_data\cn_bond_30y.feather')
print(thirty_year_bond.head())
print(thirty_year_bond.shape)
print(type(thirty_year_bond))
thirty_year_bond.columns = thirty_year_bond.columns.str.strip()
print(thirty_year_bond.columns)
thirty_year_bond['日期'] = pd.to_datetime(thirty_year_bond['日期'])
thirty_year_bond.set_index('日期', inplace=True)

          日期  cn_bond_30y
0 2009-01-05       3.6065
1 2009-01-06       3.5866
2 2009-01-07       3.5917
3 2009-01-08       3.5749
4 2009-01-09       3.5745
(4350, 2)
<class 'pandas.core.frame.DataFrame'>
Index(['日期', 'cn_bond_30y'], dtype='object')


In [201]:
df2 = pd.read_feather(r'C:\Users\m1309\Downloads\AStockData\raw\return_adj.feather')
print(df2.head())
print(df2.shape)
print(df2.iloc[0,0])


stock_code           000001.SZ  000002.SZ  000004.SZ  000005.SZ  000006.SZ  \
datetime                                                                     
2009-01-05 15:00:00   0.026427   0.038760   0.036517   0.032258   0.050710   
2009-01-06 15:00:00   0.060762   0.029851   0.029810   0.031250   0.040541   
2009-01-07 15:00:00  -0.030097  -0.005797   0.050000  -0.011364  -0.007421   
2009-01-08 15:00:00  -0.039039   0.005831   0.050125  -0.026820  -0.018692   
2009-01-09 15:00:00   0.026042  -0.001449   0.038186   0.043307   0.009524   

stock_code           000007.SZ  000008.SZ  000009.SZ  000010.SZ  000011.SZ  \
datetime                                                                     
2009-01-05 15:00:00   0.046296   0.022624   0.037688   0.019380   0.005141   
2009-01-06 15:00:00   0.032448   0.030974   0.038741   0.020913   0.033248   
2009-01-07 15:00:00  -0.031429   0.012875  -0.006993   0.020484   0.007426   
2009-01-08 15:00:00   0.035398   0.025424  -0.032864   0.012774

In [202]:
market_data = df2.fillna(0)
T=market_data.shape[0]
N=market_data.shape[1]
thirty_year_bond['cn_bond_30y']=(1+thirty_year_bond['cn_bond_30y']/100)**(1/365)-1
thirty_year_bond = thirty_year_bond.head(T)
# 对齐到 market_data 的每一个时间戳上：
thirty_year_bonds = thirty_year_bond['cn_bond_30y'].reindex(market_data.index, method='ffill')
# 然后按行相加：
stocks_excess_return = market_data.add(-thirty_year_bonds, axis=0)
#构造市场因子
market_excess_return=df2.mean(axis=1).add(-thirty_year_bonds, axis=0)
market_excess_return_matrix=pd.concat([market_excess_return]*N, axis=1)
print(market_excess_return_matrix.shape)
print(market_excess_return_matrix.head())

(3645, 5274)
                         0         1         2         3         4     \
datetime                                                                
2009-01-05 15:00:00  0.041414  0.041414  0.041414  0.041414  0.041414   
2009-01-06 15:00:00  0.034041  0.034041  0.034041  0.034041  0.034041   
2009-01-07 15:00:00  0.005267  0.005267  0.005267  0.005267  0.005267   
2009-01-08 15:00:00 -0.020810 -0.020810 -0.020810 -0.020810 -0.020810   
2009-01-09 15:00:00  0.027749  0.027749  0.027749  0.027749  0.027749   

                         5         6         7         8         9     ...  \
datetime                                                               ...   
2009-01-05 15:00:00  0.041414  0.041414  0.041414  0.041414  0.041414  ...   
2009-01-06 15:00:00  0.034041  0.034041  0.034041  0.034041  0.034041  ...   
2009-01-07 15:00:00  0.005267  0.005267  0.005267  0.005267  0.005267  ...   
2009-01-08 15:00:00 -0.020810 -0.020810 -0.020810 -0.020810 -0.020810  ...   
2009-01

In [203]:
reg = RollingRegressor(
    x=market_excess_return,
    y=stocks_excess_return,
    mode="single",
    fit_intercept=True
)
result=reg.fit(3642)
print(result)

BatchRegressionResult(
 alpha:
    stock_code              000001.SZ  000002.SZ  000004.SZ  000005.SZ  000006.SZ  \
    datetime                                                                        
    2009-01-05 15:00:00           NaN        NaN        NaN        NaN        NaN   
    2009-01-06 15:00:00           NaN        NaN        NaN        NaN        NaN   
    2009-01-07 15:00:00           NaN        NaN        NaN        NaN        NaN   
    2009-01-08 15:00:00           NaN        NaN        NaN        NaN        NaN   
    2009-01-09 15:00:00           NaN        NaN        NaN        NaN        NaN   
    ...                           ...        ...        ...        ...        ...   
    2023-12-25 15:00:00           NaN        NaN        NaN        NaN        NaN   
    2023-12-26 15:00:00  1.404881e-06   0.000022   0.000035  -0.000709  -0.000187   
    2023-12-27 15:00:00  1.252492e-07   0.000017   0.000039  -0.000710  -0.000180   
    2023-12-28 15:00:00 -3.601927e

In [204]:
regs=rolling_regression(market_excess_return, stocks_excess_return, 3642)
print(regs)

BatchRegressionResult(
 alpha:
    stock_code              000001.SZ  000002.SZ  000004.SZ  000005.SZ  000006.SZ  \
    datetime                                                                        
    2009-01-05 15:00:00           NaN        NaN        NaN        NaN        NaN   
    2009-01-06 15:00:00           NaN        NaN        NaN        NaN        NaN   
    2009-01-07 15:00:00           NaN        NaN        NaN        NaN        NaN   
    2009-01-08 15:00:00           NaN        NaN        NaN        NaN        NaN   
    2009-01-09 15:00:00           NaN        NaN        NaN        NaN        NaN   
    ...                           ...        ...        ...        ...        ...   
    2023-12-25 15:00:00           NaN        NaN        NaN        NaN        NaN   
    2023-12-26 15:00:00  1.404881e-06   0.000022   0.000035  -0.000709  -0.000187   
    2023-12-27 15:00:00  1.252492e-07   0.000017   0.000039  -0.000710  -0.000180   
    2023-12-28 15:00:00 -3.601927e